In [1]:
import undetected_chromedriver as uc
import re
from datetime import datetime
from lxml import html

In [2]:
from scraping_tools import get_html, scrape_property
from parsing_tools import extract_thumbs

In [3]:
urls = ['https://www.aruodas.lt/butai-vilniuje-antakalnyje-p-vileisio-g-naujas-projektas-antakalnyje-vileisio-1-3253985/',
    'https://www.aruodas.lt/butai-kaune-dainavoje-partizanu-g-parduodamas-jaukus-kambariu-butas-dainavoje-1-3326353/'
]

properties = []

for url in urls:
    properties.append(scrape_property(url))

In [4]:
properties

[{'Price': '251 200 €',
  'Address': 'Vilnius, Antakalnis, P. Vileišio g.',
  'Phone': '',
  'Broker': True,
  'Coordinates': (54.712566, 25.312947),
  'Reserved': '',
  'Date_scraped': '21/05/2023 13:29:42',
  'Description': '',
  'Misc': ['Nauja kanalizacija',
   'Nauja elektros instaliacija',
   'Sandėliukas',
   'Balkonas',
   'Vieta automobiliui',
   'Šarvuotos durys',
   'Kodinė laiptinės spyna',
   'Vaizdo kameros'],
  'Photos': ['https://aruodas-img.dgn.lt/object_62_106189723/vilnius-antakalnis-p-vileisio-g-2.jpg',
   'https://aruodas-img.dgn.lt/object_62_106189721/vilnius-antakalnis-p-vileisio-g-2.jpg',
   'https://aruodas-img.dgn.lt/object_62_106189717/vilnius-antakalnis-p-vileisio-g-2.jpg',
   'https://aruodas-img.dgn.lt/object_62_106189017/vilnius-antakalnis-p-vileisio-g-2.jpg',
   'https://aruodas-img.dgn.lt/object_62_106188985/vilnius-antakalnis-p-vileisio-g-2.jpg',
   'https://aruodas-img.dgn.lt/object_62_106189041/vilnius-antakalnis-p-vileisio-g-2.jpg',
   'https://aruo

In [5]:
html = get_html(urls[0])